##  Extracting the tweets from twitter using beautiful soup

In [ ]:
#Import Libraries
# -*- coding: utf-8 -*-
import csv
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver

In [ ]:
# Input conditions
query = input('Enter query (use %23 for hashtags, use %20 for space, use %3 for account): ')
option = input('Do you want to filter by date Y/N: ')
if option == "Y":
    start = input('enter start date(YYYY-MM-DD): ')
    end = input('enter the end date(YYYY-MM-DD): ')
    filter_by_date = '%20since%3A'+start+'%20until%3A'+end
    browser = webdriver.Chrome(r'C:\Users\Reddy Rohitha\chromedriver.exe')
    base_url = u'https://twitter.com/search?f=tweets&vertical=default&q='
    url = base_url + query + filter_by_date
else:
    browser = webdriver.Chrome(r'C:\Users\Reddy Rohitha\chromedriver.exe')
    base_url = u'https://twitter.com/search?f=tweets&vertical=default&q='
    #base_url = u'https://twitter.com/search?q='
    url = base_url + query

time.sleep(1)

In [ ]:
#Function to scroll the page
def tweet_scroller(url):

    browser.get(url)
    
    #define initial page height for 'while' loop
    lastHeight = browser.execute_script("return document.body.scrollHeight")
    
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        #define how many seconds to wait while dynamic page content loads
        time.sleep(3)
        newHeight = browser.execute_script("return document.body.scrollHeight")
        
        if newHeight == lastHeight:
            break
        else:
            lastHeight = newHeight
            
    html = browser.page_source

    return html

In [ ]:
#Calling beautiful soup to get the tweets
soup = BeautifulSoup(tweet_scroller(url), "html.parser")

all_tweets = soup.find_all('div',{'class':'tweet'})

In [ ]:
#Getting the respective attributes
one_list = []
headers_list = ["User","Tweet_id","Timestamp","Message","Hashtags","Replies","Retweets","Likes"]
final_list =[]
final_list.append(headers_list)
for tweet in all_tweets:
    #context = tweet.find('div',{'class':'context'}).text.replace("\n"," ").strip()
    content = tweet.find('div',{'class':'content'})
    header = content.find('div',{'class':'stream-item-header'})
    footer = content.find('div',{'class':'stream-item-footer'})
    user = re.findall("@\w+",header.find('a',{'class':'account-group js-account-group js-action-profile js-user-profile-link js-nav'}).text.replace("\n"," ").strip())[0]
    time = header.find('a',{'class':'tweet-timestamp js-permalink js-nav js-tooltip'}).prettify()#.find('span').text.replace("\n"," ").strip()
    index1 = time.find('data-conversation-id')
    index2 = time.find('href')
    tweet_id= time[index1+22:index2-2]
    index3 = time.find('title')
    index4 = time.find('span')
    timestamp = time[index3+7:index4-5]
    message = content.find('div',{'class':'js-tweet-text-container'}).text.replace("\n"," ").strip()
    hashtags = " ".join(re.findall("#[a-zA-Z]+", message))
    replies = footer.find_all('span',{'class':'ProfileTweet-actionCountForPresentation'})[0].get_text()
    retweets = footer.find_all('span',{'class':'ProfileTweet-actionCountForPresentation'})[1].get_text()
    likes = footer.find_all('span',{'class':'ProfileTweet-actionCountForPresentation'})[3].get_text()

    one_list.append(user)
    one_list.append(tweet_id)
    one_list.append(timestamp)
    one_list.append(message)
    one_list.append(hashtags)
    one_list.append(replies)
    one_list.append(retweets)
    one_list.append(likes)
    final_list.append(one_list)
    one_list = []

In [ ]:
# Use text formatting to save the file based on query 
path = r'C:\Users\Reddy Rohitha\Documents\BA CAPSTONE\TWEETS\tweets_%s.csv' %query

with open(path, "a", newline='',encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter=',',lineterminator='\n',quotechar='"')
    writer.writerows(final_list)